# [6、PyTorch中搭建分类网络实例](https://www.bilibili.com/video/BV1VF411a7oz?spm_id_from=333.788.player.switch&vd_source=cdd897fffb54b70b076681c3c4e4d45d)

神经网络一般由层或者模块，模型一般定义在`torch.nn`中

In [3]:
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## 基于手写数字识别构建分类模型

In [ ]:
training_data = datasets.FashionMNIST(
    root="minist_dataset", train=True, download=True, transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="minist_dataset", train=False, download=True, transform=ToTensor()
)

train_iter = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True)
test_iter = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True)

In [9]:
# 定义 class
from torch import Tensor

class NeuralNetwork(nn.Module):
    """"""
    def __init__(self) -> None:
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() # 变成一维张量
        self.linear_relu_stack = nn.Sequential(
            # 顺序连起来
            nn.Linear(28*28, 512), # 输入特征维度，输入特征大小
            nn.ReLU(inplace=True), # 加速
            nn.Linear(512, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 10)
        )
    def forward(self, x:Tensor):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [ ]:
# 实例化模块
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
from torchsummary import summary
# https://pypi.org/project/torch-summary/
summary(model)

Layer (type:depth-idx)                   Param #
├─Flatten: 1-1                           --
├─Sequential: 1-2                        --
|    └─Linear: 2-1                       401,920
|    └─ReLU: 2-2                         --
|    └─Linear: 2-3                       262,656
|    └─ReLU: 2-4                         --
|    └─Linear: 2-5                       5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Flatten: 1-1                           --
├─Sequential: 1-2                        --
|    └─Linear: 2-1                       401,920
|    └─ReLU: 2-2                         --
|    └─Linear: 2-3                       262,656
|    └─ReLU: 2-4                         --
|    └─Linear: 2-5                       5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0

In [12]:
X = torch.rand(1, 28, 28, device=device)
logits:Tensor = model(X)
logits.argmax()

tensor(8, device='cuda:0')

## nn.Flatten

将张量展平为两个维度，只保留 start_dim、end_dim

In [15]:
x = torch.rand((5,28, 28))
print(x.shape)
x = nn.Flatten()(x)
print(x.shape)

torch.Size([5, 28, 28])
torch.Size([5, 784])


## nn.Linear

线性层，就是计算矩阵乘法
- in_features：必须，特征维度
- out_features：必须，输出样本特征维度
- bias：偏置，也就是 w@x+b中的b

In [19]:
layer = nn.Linear(in_features=28*28, out_features=128)
hidden1 = layer(x)
print(hidden1.shape, layer.weight.shape, layer.bias.shape, sep='\n')

torch.Size([5, 128])
torch.Size([128, 784])
torch.Size([128])


## nn.Relu

非线性激活单元，非线性能够增加神经网络建模能力

## nn.Sequential

一个**容器**，将一些Module作为参数，数据就会有序地经过模块算出结果

## nn.Softmax

将数字转换为概率，概率的总和为1，公式为：

$$
x = \frac{e^x}{\sum_y{e^y}}
$$

> 简单推导，为了输出概率、总和为1。这里**概率非负**，那么使用**指数**。总和为1,那么归一化。



### Softmax 详细推导

逻辑回归中，我们使用 Sigmoid 将 z 映射为概率：
$$
P(y=1|z) = \frac{1}{1+e^{-z}} = \frac{e^z}{e^z+1}=   \frac{e^z}{e^z+e^0}
$$


#### 1. 回顾逻辑回归 (二分类)

仔细观察 $P(y=1 | z)$ 的形式，它其实是两个值的归一化：$e^z$ 和 $e^0=1$。我们可以把二分类任务看作是两个类别，一个分数为 $z$，另一个分数为 $0$。那么 Sigmoid 就是 Softmax 在 $K=2$ 时的特例。

$$
\text{Softmax}(z_1, z_2=0)_1 = \frac{e^{z_1}}{e^{z_1} + e^{z_2}} = \frac{e^{z_1}}{e^{z_1} + e^{0}} = \frac{e^{z_1}}{e^{z_1} + 1} = \frac{1}{1 + e^{-z_1}} = \text{Sigmoid}(z_1)
$$
现在考虑 $K$ 个互斥类别，这是一个**多项分布 (Multinomial Distribution)**。假设对于一个输入样本 $\mathbf{x}$，其真实类别为第 $i$ 类。模型预测的各个类别的概率为 $p_1, p_2, \dots, p_K$，其中 $p_j = P(y=j | \mathbf{x})$。

我们的目标是找到一组模型参数 $\theta$，使得对于给定的训练数据集，观测到真实标签的**似然 (Likelihood)** 最大。对于单个样本 $\mathbf{x}$，其真实标签是 $i$，那么似然函数就是 $L(\theta) = p_i$。

为了方便计算，我们通常最大化**对数似然 (Log-Likelihood)**：
$\log L(\theta) = \log p_i$

我们希望将模型的输出分数 $z_1, z_2, \dots, z_K$ 与概率 $p_1, p_2, \dots, p_K$ 联系起来。一个简单而有效的建模方式是假设对数概率 $\log p_j$ 与分数 $z_j$ 成正比：

$$
\log p_j \propto z_j \implies \log p_j = z_j - C
$$

这里的 $C$ 是一个常数，因为它不依赖于类别 $j$，所以对于所有类别都是一样的。这个常数 $C$ 的作用是确保所有概率之和为 1，即起到归一化的作用。

从上式可得：
$$
p_j = e^{z_j - C} = e^{z_j} \cdot e^{-C}
$$

因为所有类别的概率之和必须为 1：
$$
\sum_{j=1}^{K} p_j = \sum_{j=1}^{K} e^{z_j} \cdot e^{-C} = 1
$$

$$
e^{-C} \left( \sum_{j=1}^{K} e^{z_j} \right) = 1 \implies e^{-C} = \frac{1}{\sum_{j=1}^{K} e^{z_j}}
$$

将 $e^{-C}$ 代回到 $p_j = e^{z_j} \cdot e^{-C}$ 的表达式中，我们得到：

$$
p_j = \frac{e^{z_j}}{\sum_{k=1}^{K} e^{z_k}}
$$

这正是 Softmax 公式。这个推导说明，Softmax 函数是源于用线性分数来建模对数概率，并通过最大似然原理自然导出的结果。

## Model Parameters
利用模型的 parameters() 或 named_parameters() 方法访问所有参数。

In [23]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")
    break

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0087, -0.0049,  0.0345,  ..., -0.0021,  0.0271,  0.0087],
        [-0.0067, -0.0071,  0.0065,  ...,  0.0173, -0.0013,  0.0086]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

